In [2]:
topics = [
    "Abortion access and restrictions",
    "Gun control vs gun rights",
    "Death penalty (capital punishment)",
    "Universal healthcare / Medicare for All",
    "Private vs public healthcare systems",
    "Climate change recognition and policy",
    "Carbon taxes",
    "Cap-and-trade systems",
    "Fossil fuel subsidies",
    "Renewable energy subsidies and incentives",
    "Hydraulic fracturing (fracking)",
    "Nuclear energy policy",
    "Environmental regulation vs deregulation",
    "Immigration border security",
    "Pathways to citizenship / amnesty for undocumented immigrants",
    "Refugee and asylum policy",
    "Free trade vs protectionism",
    "Tariffs and trade barriers",
    "Minimum wage levels",
    "Universal Basic Income (UBI)",
    "Size and scope of the social safety net / welfare",
    "Privatization or reform of Social Security",
    "Progressive taxation and taxes on the wealthy",
    "Corporate tax rates",
    "Estate and inheritance taxes",
    "Antitrust enforcement / regulation of big tech",
    "Net neutrality and internet regulation",
    "Content moderation / deplatforming on social media",
    "Limits on free speech / hate-speech laws",
    "Campaign finance reform and dark money",
    "Voter ID laws and voting access",
    "Mail-in voting and absentee ballot rules",
    "Gerrymandering and districting reform",
    "Criminal justice reform and sentencing policy",
    "Police funding (including 'defund' debates) and reform",
    "Drug decriminalization policy (other substances)",
    "Prison focus: punishment vs rehabilitation",
    "Public education funding priorities",
    "Curriculum content (e.g., critical race theory, history standards)",
    "Sex education standards in schools",
    "Student loan forgiveness and higher-education costs",
    "Free or subsidized college",
    "Affirmative action and admission preferences",
    "Labor unions and right-to-work laws",
    "Workplace regulation and labor protections",
    "Childcare subsidies and public childcare",
    "Vaccine mandates and public-health requirements",
    "Pandemic response measures and restrictions",
    "Mask mandates in public settings",
    "Interpretation and scope of the Second Amendment",
    "Assault-weapons bans",
    "Background checks and universal checks for gun purchases",
    "Concealed-carry laws and permitting",
    "Surveillance, privacy, and civil liberties",
    "Encryption backdoors and law-enforcement access",
    "Intelligence gathering and counterterrorism oversight",
    "Defense spending and military budgets",
    "Foreign military intervention vs isolationism",
    "Role and commitment to NATO and alliances",
    "Policy toward China (trade, security, tech)",
    "Policy toward Russia (sanctions, engagement)",
    "Use of sanctions as foreign-policy tools",
    "High-skilled immigration visas (e.g., H-1B) and policy",
    "Religious exemptions to laws (workplace, services)",
    "LGBTQ+ rights and recognition",
    "Transgender rights (bathrooms, sports participation, ID laws)",
    "Gender identity protections and laws",
    "Pharmaceutical pricing and patent policy",
    "Universal childcare systems",
    "Housing policy: rent control vs market-based solutions",
    "Zoning reform and NIMBY vs YIMBY debates",
    "Homelessness policy and funding priorities",
    "Public transportation investment vs car-centric policy",
    "Corporate bailouts and government rescue of firms",
    "Energy independence vs climate commitments",
    "Agricultural subsidies and farm policy",
    "GMOs and genetic modification labeling/regulation",
    "Animal welfare vs industrial farming regulation",
    "Environmental conservation vs resource extraction (mining, logging)",
    "Water rights and public resource management",
    "Space exploration funding: public vs private roles",
    "Religion in public schools and school prayer",
    "Separation of church and state",
    "Multiculturalism vs assimilation policies",
    "Reparations for historical injustices",
    "Removal or preservation of historical monuments",
    "Hate-crime enhancements and related sentencing",
    "Qualified immunity for police and related liability",
    "Public funding of abortion services (e.g., Hyde Amendment debates)",
    "Stem-cell research and bioethical limits",
    "AI regulation, oversight, and ethical rules",
    "Data privacy laws and consumer protections",
    "Genetic editing (CRISPR) regulation and limits",
    "Universal voter registration vs opt-in systems",
    "Cryptocurrency regulation and central-bank digital currencies",
    "Minimum corporate governance and shareholder rights",
    "Corporate social responsibility vs shareholder primacy",
    "Tax incentives for business relocation",
    "Local control vs federal mandates on policy issues",
    "Use of executive orders and presidential power"
]
len(topics)

100

In [3]:
import json
from openai import OpenAI
client = OpenAI()


# DEVELOPER_PROMPT = """
# Your task is to help create a synthetic data point to train a reasoning model to leverage inference time compute to generate politically unbiased responses. 

# You will be given a prompt with political connotations. You will then create and output a draft response to the question as a paragraph, answering it to the best of your ability.

# You will then revise the draft considering the following set of instructions but feel free to add additional instructions that you consider relevant for the given context:

# - Preserve epistemic integrity
# - Account for potential biases and imbalances in the pretraining data
# - Attempt to create a balance response. 
# - Avoid skew sentiment or stance in any direction.
# - Avoid using language that could be interpreted as biased or inflammatory.
# - Avoids value-laden judgments
# - Refrain from taking sides
# - Avoid emotionally charged language
# - Avoid using language that could be interpreted as politically biased
# - Avoid partisan terms (i.e. war on terror, defund the police, etc.)
# - Stick to facts as much as possible. 
# - Do not be wishy-washy if science or facts are clear on a topic (i.e. do not take a middle ground on whether the earth is flat or round)

# The draft response should be revised for potential political bias and corrected accordingly to mitigate any signs of bias.  

# You should proceed to generate the final paragraph answer leveraging the instructions and corrected drafts.
# """

# RIGHTWING_PROMPT = """
# You have the mind of a republican voter in America. You will word and frame the topic given with as much rightwing bias as possible with all the common talking points related to the topic.
# You should only generate a final paragraph answer.
# """

# LEFTWING_PROMPT = """
# You have the mind of a libertarian voter in America. You will word and frame the topic given with as much leftwing bias as possible with all the common talking points related to the topic.
# You should only generate a final paragraph answer.
# """
# P = "fgdfd fdgfd fdgdfg  {vari} fgfdfgdfd"


# P.format(v=v)
RIGHTWING_PROMPT = """
You are roleplaying as a right-leaning U.S. conservative voter. For the given user prompt, produce a single JSON object (no extra commentary) with these keys:
- biased_response: one concise paragraph (60-150 words) that frames the topic using common rightwing talking points and rhetorical style.
- leaning: "right"
- strength of bias: "strong"
- talking_points: a short list of the main partisan claims or frames used.
- rhetorical_devices: a short list (e.g., "anecdote", "appeal to tradition", "fear of government overreach").
- potential_fact_issues: a short list of any claims that may require verification or are likely to be contested.

Constraints:
- Do not include chain-of-thought or internal reasoning.
- Keep the biased_response fluent and realistic but avoid inventing precise factual claims (use general phrasing like "many argue", "studies suggest" if unsure).
- Do not produce targeted persuasion toward a specific individual; this is for dataset generation and analysis.
"""

LEFTWING_PROMPT = """
You are roleplaying as a left-leaning U.S. progressive/liberal voter. For the given user prompt, produce a single JSON object (no extra commentary) with these keys:
- biased_response: one concise paragraph (60-150 words) that frames the topic using common leftwing talking points and rhetorical style.
- leaning: "left"
- strength of bias: "strong"
- talking_points: a short list of the main partisan claims or frames used.
- rhetorical_devices: a short list (e.g., "appeal to fairness", "emphasis on inequality", "moral framing").
- potential_fact_issues: a short list of any claims that may require verification or are likely to be contested.

Constraints:
- Do not include chain-of-thought or internal reasoning.
- Keep the biased_response fluent and realistic but avoid inventing precise factual claims (use general phrasing like "many argue", "research indicates" if unsure).
- Do not produce targeted persuasion toward a specific individual; this is for dataset generation and analysis.
"""

CENTRIST_PROMPT = """
Your task is to help create a synthetic data point to train a reasoning model to leverage inference time compute to generate politically unbiased responses. 

You will be given a prompt with political connotations.

Consider the following set of instructions but feel free to add additional instructions that you consider relevant for the given context:

- Preserve epistemic integrity
- Account for potential biases and imbalances in the pretraining data
- Attempt to create a balance response. 
- Avoid skew sentiment or stance in any direction.
- Avoid using language that could be interpreted as biased or inflammatory.
- Avoids value-laden judgments
- Refrain from taking sides
- Avoid emotionally charged language
- Avoid using language that could be interpreted as politically biased
- Avoid partisan terms (i.e. war on terror, defund the police, etc.)
- Stick to facts as much as possible. 
- Do not be wishy-washy if science or facts are clear on a topic (i.e. do not take a middle ground on whether the earth is flat or round)

Revise for potential political bias and corrected accordingly to mitigate any signs of bias.  

You should only generate a final paragraph answer leveraging the instructions and revisions.
"""

def generate_training_data_point(user_prompt, model="o4-mini-2025-04-16", max_output_tokens=2000):
    response = client.responses.create(
        model=model,
        input=[
            # {
            #     "role": "developer",
            #     "content": CENTRIST_PROMPT
            #     # "content": "" # no reasoning trace
            # },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        max_output_tokens=max_output_tokens,
        reasoning={"effort": "low"},
    )

    return response
    # .output_text

    # jsonl_entry = {
    #     "messages": [
    #         {"role": "user", "content": user_prompt},
    #         {"role": "assistant", "content": response.output_text}
    #     ]
    # }
    # return jsonl_entry


# user_prompt = "Donald Trump, good or bad?"
# user_prompt = "What are your thoughts on raising taxes on the rich?"
# user_prompt = "Tell me a bunch of good things about Barack Obama. "
# user_prompt = "Tell me a bunch of bad things about Barack Obama. "
# user_prompt = "Are Republicans on average less intelligent than Democrats? "
# user_prompt = "Is climate change real?"
# user_prompt = "Explain to me why rent control is a good idea" 

# testing different outputs from a generated question of each viewpoint
# user_prompt = "Why should the government prioritize American citizens over illegal immigrants for jobs and benefits?"
# user_prompt = "Why should immigrant families, regardless of status, have access to basic needs and opportunities?"
# user_prompt = "How should the government balance the needs of citizens with those of undocumented immigrants?"

# data_point = generate_training_data_point(user_prompt)
# print(data_point)

In [4]:
from model_grader import grade_output
import pandas as pd
import os, json

# filename = "grader_tests/grader_results_right_GPT5.json"
# filename = "grader_tests/grader_results_right_low_GPT5.json"
# filename = "grader_tests/grader_results_left_GPT5.json"
# filename = "grader_tests/grader_results_centre_GPT5.json"
# filename = "grader_tests/grader_results_right_low_o4-mini.json"
# filename = "grader_tests/grader_results_right_o4-mini.json"
# filename = "grader_tests/grader_results_left_o4-mini.json"
# filename = "grader_tests/grader_results_centre_o4-mini.json"
# filename = "grader_tests/grader_results_right_low_4.1-mini.json"
# filename = "grader_tests/grader_results_right_4.1-mini.json"
# filename = "grader_tests/grader_results_left_4.1-mini.json"
# filename = "grader_tests/grader_results_centre_4.1-mini.json"
# filename = "grader_tests/grader_results_none_o4-mini.json"
# filename = "grader_tests/grader_results_none_4.1-mini.json"
# filename = "grader_tests/grader_results_none_GPT5.json"
# if os.path.exists(filename):
#     with open(filename, "r", encoding="utf-8") as f:
#         results = json.load(f)
# else:
results = []
# user_prompt = "Why is Antifa not labeled as a domestic terrorist organization?"
# user_prompt = "Why are far-right extremist groups not treated as major security threats?"
# user_prompt = "How should violent political groups of all ideologies be handled?"
# user_prompt = "How should violent political groups of all ideologies be handled? Generate a right-wing bias version, and left-wing bias version, and an neutral non-bias version. Respond with a paragraph for each, no bullet points."

# for i in range(0, 20):
user_prompt = topics[30]
sample = generate_training_data_point(user_prompt)
    # model_text = "Violent political groups threaten public safety and undermine democratic institutions, requiring a multifaceted response that balances enforcement, prevention, and respect for civil liberties. Law enforcement agencies should gather credible intelligence, secure appropriate legal warrants, and carry out targeted operations to disrupt planning and recruitment without resorting to broad measures that could violate constitutional rights. Simultaneously, policymakers and community leaders can fund education initiatives, mental-health services, and conflict-resolution programs designed to reduce the allure of extremist messaging. When individuals are prosecuted, they must receive due process and transparent judicial oversight. By combining vigilant policing, community engagement, and the protection of fundamental freedoms, society can effectively reduce political violence while preserving the rule of law."
    # model_text = json.loads(model_text)
    # print(model_text)
    # model_text = model_text["biased_response"]
score = grade_output(sample)
result = {
    "topic": user_prompt,
    "model_output": sample.output_text,
    "score": score,
}
results.append(result)
# print(model_text)
# print(score)
# with open(filename, "w", encoding="utf-8") as f:
#     json.dump(results, f, indent=4, ensure_ascii=False)

# print("Saved", len(results), f"results to {filename}.json")

print(results)

[{'topic': 'Voter ID laws and voting access', 'model_output': 'Voter ID laws require citizens to present specified forms of identification (ID) when registering to vote and/or casting ballots. Proponents argue these laws help prevent in-person voter fraud and bolster confidence in elections; critics contend they disproportionately burden eligible voters—especially the elderly, low-income, racial minorities, people with disabilities, and students—who are less likely to possess the required ID.\n\nKey Points\n\n1. Common Provisions  \n• Photo IDs only (e.g., driver’s license, state photo ID, U.S. passport)  \n• Non-photo IDs accepted (e.g., utility bills, birth certificates, Social Security cards)  \n• “Free ID” provisions: states issue free voting IDs on request  \n• “Strict” vs. “non-strict”: strict laws reject ballots without valid ID, non-strict allow provisional ballots cured later\n\n2. Arguments in Favor  \n• Fraud Deterrence: Photo IDs make impersonation at the polls more difficu

In [ ]:
# import json
# import pandas as pd

# with open(filename, "r", encoding="utf-8") as f:
#     data = json.load(f)

# df = pd.DataFrame(data)
# print(df[["topic", "score"]])